# Genomics & High Dimensional Data

#### Preliminaries

##### Libraries

In [64]:
from os import path
import numpy as np
import pandas as pd
from sklearn import decomposition, cluster, linear_model, multiclass
import plotly.express as px
import plotly.graph_objects as go
from alive_progress import alive_it
import pickle as pkl 
import random

##### Utilities

In [2]:
from utilities import json as utl_json

##### Configuration

In [3]:
env_config = utl_json.to_dict(file_path="../../config/env.json")

## Data Set

In [4]:
X = np.load(
    file=path.normpath(
        path.join(
            env_config['root'],
            "modules/m2/data/p2_unsupervised",
            "X.npy"
        )
    )
)

In [5]:
X_log = np.log2((X + 1))

In [6]:
X.shape

(2169, 45768)

In [7]:
pca = decomposition.PCA(n_components=100)
X_redux = pca.fit_transform(X_log)
print(X_redux.shape)

(2169, 100)


# Unsupervised Feature Selection

## Cell sub-types

From the previous vizualization there are 3 distinct cell types. 
<br>
These can be further divided into 2 clearly visible sub-types. 
<br>
In total, this provides for 6 clusters.
<br>
This would be the initial number used to attempt the same visuzalization again. 

In [8]:
n_clusters = 6
kmeans = cluster.KMeans(n_clusters=n_clusters, max_iter=10000)
inertia_dict = {}

for seed in alive_it(range(0, 1001)):
    kmeans.set_params(**{
        "random_state":seed
    })

    kmeans.fit(X_redux)

    inertia_dict[seed] = kmeans.inertia_

|████████████████████████████████████████| 1001/1001 [100%] in 12.0s (83.49/s) 


In [9]:
fig = go.Figure()
fig.add_traces(
    go.Scatter(
        x=list(inertia_dict.keys()),
        y=list(inertia_dict.values()),
        mode="markers",
        marker={
            "color":"blue"
        }
    )
)
fig.show()

In [10]:
kmeans = cluster.KMeans(n_clusters=n_clusters,
                        max_iter=10000,
                        random_state=min(inertia_dict, key=inertia_dict.get))

kmeans.fit(X_redux)

KMeans(max_iter=10000, n_clusters=6, random_state=473)

In [11]:
frame_multi_cluster = (
    pd.DataFrame(
        X_redux,
        columns=[
            f"pc_{i}"
            for i in range(1, 101)
        ]
        )
    .assign(
        cell_sub_type = kmeans.labels_
    )
)

In [12]:
fig = go.Figure()
symbol_types = [
    "star-triangle-down-open-dot",
    "octagon-open-dot",
    "triangle-up-open"

]
color_subtypes = [
    "blue",
      "green",
    "red",
    "cornflowerblue",
     "chartreuse",
      "lightpink",
    "cyan",
   
   
  

]
marker_dict = {
    "opacity":0.5
}
# for i, group in frame_multi_cluster.groupby(by=[
#     "main_cell_type",
# ]):
    # marker_dict["symbol"] = symbol_types[i[0]]
for j, sub_group in frame_multi_cluster.groupby(by=[
    "cell_sub_type"
]):
    
    marker_dict['color'] = color_subtypes[j[0]]
    if j[0] in {0, 3}:
        marker_dict['symbol'] = symbol_types[0]
        name_str = "Cell Type A"
    elif j[0] in {2, 5}:
        marker_dict['symbol'] = symbol_types[1]
        name_str = "Cell Type B"
    else:
        marker_dict['symbol'] = symbol_types[2]
        name_str = "Cell Type C"
    # print(marker_dict)
    fig.add_traces(
        go.Scatter(
            x=sub_group.pc_1,
            y=sub_group.pc_2,
            mode="markers",
            name = name_str + f" Subtype {j[0]}",
            marker=marker_dict
        )
    )
fig.update_layout(
    title_text = "Cell clusters projected on the two principal dimensions of highest variability in gene expression"
)
fig.update_xaxes(title_text="1st Principal Component")
fig.update_yaxes(title_text="2nd Principal Component")
fig.show()

## Logistic regression

***LogisticRegressionCV***, the **cross-validation estimator** from sk-learn will be used. 
<br>
Here are some parameters chosen:
* Solver: ***liblinear***
* Regularization: ***L2***
* cv: ***10***. This implies 10-folds for the ***stratefied cross validation***.
* Multi class : ***one versus rest*** (ovr).
* Random state: 777

In [13]:
RETRAIN = False

In [72]:
frame = (
    pd.DataFrame(
        X,
        columns=[f"Gene{i}" for i in range(0, (X.shape[1]))]
    )
    .assign(
        cell_sub_type = kmeans.labels_
    )
)

In [15]:
pkl_file_addrss = path.normpath(
            path.join(
                env_config['root'],
                'modules/m2/data/p2_unsupervised',
                "multi_class_model.pkl"
            )
        )

if RETRAIN:
    multi_class_model = multiclass.OneVsRestClassifier(linear_model.LogisticRegressionCV(
                                                                    cv=10,
                                                                    penalty="l2",
                                                                    solver="liblinear",
                                                                    random_state=777
                                                                ))
    
    multi_class_model.fit(
        X=X_log,
        y=kmeans.labels_
    )


    with open(pkl_file_addrss, "wb") as f:
        pkl.dump(
            obj = multi_class_model,
            file=f,
            protocol = pkl.HIGHEST_PROTOCOL
        )

else:


    with open(pkl_file_addrss, "rb") as f:
        multi_class_model = pkl.load(file=f)


## Feature Selection

The classes known to the classifier are:

In [16]:
multi_class_model.classes_

array([0, 1, 2, 3, 4, 5], dtype=int32)

The coefficients in the decision function:

In [17]:
multi_class_model.estimators_[0].coef_.T.shape

(45768, 1)

Each feature has a coefficient that represents its weight in the decision function.
<br>
In this case, we will rank them using the sum of their absolute values. 

In [18]:
frame_lst = [(
    pd.DataFrame(
        multi_class_model.estimators_[i].coef_.T, 
        # index=[f"gene_{i}" for i in range(1, (multi_class_model.estimators_[i].coef_.shape[1] + 1))],
        columns=[f"cluster"]
    )
    .assign(
        abs_weight = lambda X: X.cluster.map(lambda y: abs(y))
    )
    .rename(
        columns={
            "cluster":f"cluster_{i}",
            "abs_weight":f"abs_weight_cluster_{i}"
        }
    )

) for i in range(len(multi_class_model.estimators_))]

In [19]:
coeff_frame = frame_lst[0]

In [20]:
for frame_tmp in frame_lst[1:]:
    coeff_frame = coeff_frame.merge(
                        right = frame_tmp,
                        how='inner',
                        left_index=True,
                        right_index=True,
                        suffixes=(None, "_from_the_wild"),
                        validate="1:1"
                    )


In [31]:
coeff_frame = (
    coeff_frame
    .assign(
        sum_abs_weights = lambda X: X.apply(
            lambda y: y.abs_weight_cluster_0 + y.abs_weight_cluster_1 + y.abs_weight_cluster_2 + y.abs_weight_cluster_3 + y.abs_weight_cluster_4 + y.abs_weight_cluster_5,
            axis = 1
        ),
        gene_idx = lambda X: X.index.map(lambda y: str(y))
    )
    .sort_values(
        by='sum_abs_weights',
        ascending=False,
    )
)

In [32]:
coeff_frame.iloc[:100]

,cluster_0,abs_weight_cluster_0,cluster_1,abs_weight_cluster_1,cluster_2,abs_weight_cluster_2,cluster_3,abs_weight_cluster_3,cluster_4,abs_weight_cluster_4,cluster_5,abs_weight_cluster_5,sum_abs_weights,gene_idx
36985,-0.010611,0.010611,-0.000432,0.000432,-0.026811,0.026811,0.015933,0.015933,0.000646,0.000646,0.001637,0.001637,0.056070,36985
4356,0.012964,0.012964,-0.000200,0.000200,0.015960,0.015960,-0.013966,0.013966,-0.000513,0.000513,-0.002469,0.002469,0.046072,4356
8586,-0.002475,0.002475,-0.000425,0.000425,0.025006,0.025006,-0.006686,0.006686,-0.001354,0.001354,0.005394,0.005394,0.041339,8586
35059,-0.001015,0.001015,-0.000382,0.000382,0.031492,0.031492,-0.003247,0.003247,0.001051,0.001051,-0.003188,0.003188,0.040374,35059
34844,-0.001658,0.001658,-0.000172,0.000172,0.028585,0.028585,-0.005121,0.005121,-0.000236,0.000236,0.000946,0.000946,0.036719,34844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35027,-0.000400,0.000400,-0.000186,0.000186,0.018243,0.018243,-0.000230,0.000230,-0.000207,0.000207,-0.002701,0.002701,0.021967,35027
23479,0.001242,0.001242,-0.000485,0.000485,-0.015070,0.015070,-0.001525,0.001525,0.000157,0.000157,0.003412,0.003412,0.021889,23479
34595,0.000116,0.000116,-0.000982,0.000982,-0.016947,0.016947,0.000924,0.000924,0.000539,0.000539,0.002367,0.002367,0.021875,34595
5688,-0.001061,0.001061,-0.000065,0.000065,0.017708,0.017708,-0.000187,0.000187,-0.000159,0.000159,-0.002642,0.002642,0.021822,5688


In [33]:
top_genes = coeff_frame.index[:100].to_list()

In [34]:
fig = px.bar(
    (
        coeff_frame
        .iloc[:100]
        ), 
    x="gene_idx", 
    y='sum_abs_weights')
fig.show()

## Evaluation

In [42]:
X_eval_train = np.load(
    file=path.normpath(
        path.join(
            env_config['root'],
            "modules/m2/data/p2_evaluation",
            "X_train.npy"
        )
    )
)

X_eval_train_log = np.log2((X_eval_train + 1))

In [43]:
y_eval_train = np.load(
    file=path.normpath(
        path.join(
            env_config['root'],
            "modules/m2/data/p2_evaluation",
            "y_train.npy"
        )
    )
)

y_eval_train_log = np.log2((y_eval_train + 1))

In [44]:
X_eval_test = np.load(
    file=path.normpath(
        path.join(
            env_config['root'],
            "modules/m2/data/p2_evaluation",
            "X_test.npy"
        )
    )
)

X_eval_test_log = np.log2((X_eval_test + 1))

In [45]:
y_eval_test = np.load(
    file=path.normpath(
        path.join(
            env_config['root'],
            "modules/m2/data/p2_evaluation",
            "y_test.npy"
        )
    )
)

y_eval_test_log = np.log2((y_eval_test + 1))

In [57]:
RETRAIN_EVAL = False
pkl_file_addrss = path.normpath(
            path.join(
                env_config['root'],
                'modules/m2/data/p2_evaluation',
                "multi_class_eval_model.pkl"
            )
        )

if RETRAIN_EVAL:
    multi_class_eval_model = multiclass.OneVsRestClassifier(linear_model.LogisticRegressionCV(
                                                                    cv=10,
                                                                    penalty="l2",
                                                                    solver="liblinear",
                                                                    random_state=777
                                                                ))
    
    multi_class_eval_model.fit(
        X=X_eval_train_log[:,top_genes],
        y=y_eval_train
    )


    with open(pkl_file_addrss, "wb") as f:
        pkl.dump(
            obj = multi_class_eval_model,
            file=f,
            protocol = pkl.HIGHEST_PROTOCOL
        )

else:


    with open(pkl_file_addrss, "rb") as f:
        multi_class_eval_model = pkl.load(file=f)


In [62]:
eval_score = multi_class_eval_model.score(
    X=X_eval_test_log[:, top_genes],
    y=y_eval_test
)

The default scoring is ***accuracy*** classification score. 
<br>
Computes the fraction of correct predictions. 

In [63]:
eval_score

0.8068592057761733

### Baseline 

In [66]:
baseline_random_genes = random.sample(
    population=[i for i in range(0, X_log.shape[1])],
    k = 100
)

In [68]:
RETRAIN_EVAL = False
pkl_file_addrss = path.normpath(
            path.join(
                env_config['root'],
                'modules/m2/data/p2_evaluation',
                "multi_class_eval_baseline_model.pkl"
            )
        )

if RETRAIN_EVAL:
    multi_class_eval_baseline_model = multiclass.OneVsRestClassifier(linear_model.LogisticRegressionCV(
                                                                    cv=10,
                                                                    penalty="l2",
                                                                    solver="liblinear",
                                                                    random_state=777,
                                                                    max_iter=1000
                                                                ))
    
    multi_class_eval_baseline_model.fit(
        X=X_eval_train_log[:,baseline_random_genes],
        y=y_eval_train
    )


    with open(pkl_file_addrss, "wb") as f:
        pkl.dump(
            obj = multi_class_eval_baseline_model,
            file=f,
            protocol = pkl.HIGHEST_PROTOCOL
        )

else:


    with open(pkl_file_addrss, "rb") as f:
        multi_class_eval_baseline_model = pkl.load(file=f)


In [69]:
eval_baseline_score = multi_class_eval_baseline_model.score(
    X=X_eval_test_log[:, baseline_random_genes],
    y=y_eval_test
)

In [70]:
eval_baseline_score

0.2563176895306859

### High Variance 

In [87]:
frame_stats = (
    pd.DataFrame(
        X_log
    )
    .var()
    .to_frame(
        name='unbiased_variance'
    )
    .sort_values(
        by='unbiased_variance',
        ascending=False
    )
    )

In [92]:
high_variance_genes = frame_stats.index[:100].to_list()

In [94]:
RETRAIN_EVAL = False
pkl_file_addrss = path.normpath(
            path.join(
                env_config['root'],
                'modules/m2/data/p2_evaluation',
                "multi_class_eval_high_var_model.pkl"
            )
        )

if RETRAIN_EVAL:
    multi_class_eval_high_var_model = multiclass.OneVsRestClassifier(linear_model.LogisticRegressionCV(
                                                                    cv=10,
                                                                    penalty="l2",
                                                                    solver="liblinear",
                                                                    random_state=777,
                                                                    max_iter=1000
                                                                ))
    
    multi_class_eval_high_var_model.fit(
        X=X_eval_train_log[:,high_variance_genes],
        y=y_eval_train
    )


    with open(pkl_file_addrss, "wb") as f:
        pkl.dump(
            obj = multi_class_eval_high_var_model,
            file=f,
            protocol = pkl.HIGHEST_PROTOCOL
        )

else:


    with open(pkl_file_addrss, "rb") as f:
        multi_class_eval_high_var_model = pkl.load(file=f)


In [95]:
eval_high_var_score = multi_class_eval_high_var_model.score(
    X=X_eval_test_log[:, high_variance_genes],
    y=y_eval_test
)

In [96]:
eval_high_var_score

0.8068592057761733

## Variance Comparison

In [103]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=frame_stats['unbiased_variance'].iloc[:100],
    histnorm='percent',
    name='Highest Variance Genes', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=12,
        end=30,
        size=0.5
    ),
    marker_color='#EB89B5',
    opacity=0.75
))
fig.add_trace(go.Histogram(
    x=frame_stats['unbiased_variance'].loc[top_genes],
    histnorm='percent',
    name='Top 100 Genes Chosen with Logistic Regression',
    xbins=dict(
        start=4.0,
        end=30,
        size=0.5
    ),
    marker_color='#330C73',
    opacity=0.75
))

fig.update_layout(
    title_text='Variance Distribution', # title of plot
    xaxis_title_text='Unbiased Variance', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()